In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
# Skip this cell if CSR2D/ is already added under PYHTONPATH 
import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)  

# Function definitions

In [2]:
import mpmath as mp
import numpy as np
import cupy as cp
import sys

import matplotlib.pyplot as plt

from scipy.ndimage import convolve as conv
from scipy.signal import convolve2d, fftconvolve, oaconvolve
from cupyx.scipy.ndimage import convolve as cupy_conv

from csr2d.core import psi_s, psi_x, alpha, psi_x_ss
from csr2d.dist import lambda_p_Gauss

In [4]:
psi_x_ss(0.02,0.03,0.999)

-2.173880284563949

In [5]:
psi_x(0.02,0.03,0.999)

-2.173880284563949

In [ ]:
def is_even(n):
    if n % 2 == 0:
        return -1
    else:
        return 0
a = np.array([[1, 2, 0, 0,2],[5, 3, 0, 4,5],[0, 0, 0, 7,3],[9, 3, 0, 0,2]])
(is_even(a.shape[0]),is_even(a.shape[1]))

In [5]:
def Ws(gamma, rho, sigmaz, sigmax, dz, dx, method='oa', xp=np, cupy_conv=None):
    """
    Apply 2D convolution to compute the longitudinal wake Ws on a grid 
    Also returns the zvec and xvec which define the grid
    
    Still needs to improve the convolution step
    """
    beta = (1-1/gamma**2)**(1/2)
    if method == 'cupy':
        pass
    elif method == 'oa':
        conv_method = oaconvolve
    elif method == 'fft':
        conv_method = fftconvolve
    elif method == '2d':
        conv_method = convolve2d
    else:
        print('Invalid convolution method specified!!')
        sys.exit("Exiting...")
    
    print('Here A')
    # Creating the grid of the z-derivative of the bunch distribution
    zvec = np.arange(-5*sigmaz, 5*sigmaz, dz)
    xvec = np.arange(-5*sigmax, 5*sigmax, dx)
    lambdap_list = [[lambda_p_Gauss(i,j) for j in xvec] for i in zvec] 
    lambdap_grid = xp.array(lambdap_list,dtype=float)
    
    print('Here B')
    # Creating the grid of the potential 
    zvec2 = np.arange(-10*sigmaz, 10*sigmaz, dz)
    xvec2 = np.arange(-10*sigmax, 10*sigmax, dx)
    psi_s_list = [[psi_s(i/2/rho,j,beta) for j in xvec2] for i in zvec2]   
    psi_s_grid = xp.array(psi_s_list,dtype=float)
    
    print('Here C')
    # Convolution
    if method == 'cupy':
        #origin = (is_even(psi_s_grid.shape[0]),is_even(psi_s_grid.shape[1]))
        #conv_s = cupy_conv(lambdap_grid, psi_s_grid, mode='constant',origin=origin)
        conv_s = cupy_conv(lambdap_grid, psi_s_grid, mode='constant')
    else:
        #conv_s = conv_method(lambdap_grid, psi_s_grid, mode='same', boundary='fill', fillvalue=0)
        conv_s = conv_method(lambdap_grid, psi_s_grid, mode='same')
        
    WsConv = (beta**2/rho)*(conv_s)*(dz*dx)
   
    return zvec, xvec, WsConv

# Testing

In [8]:
gamma = 500
beta = (1-1/gamma**2)**(1/2)
rho = 1
sigmax = 10E-6
sigmaz = 10E-6

# Adjust the step size here
# Small steps might greatly increase computation time...
#dz = 0.1*sigmaz
#dx = 0.1*sigmax

N = 4
dz = 1.0E-4/N
dx = 1.0E-4/N

### Testing how to make psi_s_list faster...

In [9]:
%%time
zvec2 = np.arange(-10*sigmaz, 10*sigmaz, dz)
xvec2 = np.arange(-10*sigmax, 10*sigmax, dx)

CPU times: user 26 µs, sys: 0 ns, total: 26 µs
Wall time: 29.3 µs


In [ ]:
# Looping is SLOW, use vectorization in the future
psi_s_list = [[psi_s(i/2/rho,j,beta) for j in xvec2] for i in zvec2]  
psi_s_grid = np.array(psi_s_list,dtype=float)

In [10]:
zm, xm = np.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

In [11]:
xm[0][0].__class__

numpy.float64

In [16]:
psi_x_ss(zm, xm, beta)

array([[ 2.00014178,  2.00011292,  2.00008091,  2.0000478 ,  1.99935015,
         1.99997363,  1.99993406,  1.99989847],
       [ 2.00013052,  2.0001074 ,  2.0000784 ,  2.00004616,  1.99688248,
         1.99997275,  1.9999342 ,  1.99990121],
       [ 2.00009999,  2.00009345,  2.0000731 ,  2.00004419,  1.999347  ,
         1.99997112,  1.99993564,  1.99991073],
       [ 1.99994126,  2.00002402,  2.00004999,  2.00003836,  1.97714054,
         1.99997082,  1.99995018,  1.99996783],
       [-1.01429801, -1.01869133, -1.02704441, -1.04898133,  0.        ,
        -3.43325217, -3.7395658 , -3.83105572],
       [-1.92238896, -1.94181626, -1.96075743, -1.97921739, -1.99733791,
        -2.01470578, -2.03174658, -2.04833077],
       [-1.95209408, -1.9639184 , -1.97555182, -1.98699628, -1.99838986,
        -2.00931502, -2.02019243, -2.03088712],
       [-1.96375547, -1.9726506 , -1.98143518, -1.99011051, -1.99881389,
        -2.00712698, -2.01546952, -2.02370559]])

In [ ]:
from itertools import chain
def fun(i):
    return tuple(psi_s(i/2/rho,j,beta) for j in xvec2)

In [ ]:
%%time  
## This iterable method seems equally slow...
a = np.fromiter(chain.from_iterable(fun(i) for i in zvec2), float)
a.shape = len(zvec2),len(xvec2)

### Trying with CP

In [17]:
zvec2 = cp.arange(-10*sigmaz, 10*sigmaz, dz)
xvec2 = cp.arange(-10*sigmax, 10*sigmax, dx)
zm, xm = cp.meshgrid(zvec2, xvec2, sparse=False, indexing='ij')

CUDARuntimeError: cudaErrorNoDevice: no CUDA-capable device is detected

## single run testing

In [ ]:
%%time
zvec, xvec, WsConv = Ws(gamma, rho, sigmaz, sigmax, dz, dx, method='cupy', xp=cp, cupy_conv=cupy_conv)

In [ ]:
# Checking the result is a cupy entity
WsConv.__class__

In [ ]:
temp=abs(zvec)
zix_0 = np.where(temp == temp.min())[0][0] 
temp=abs(zvec-1*sigmaz)
zix_1sig = np.where(temp == temp.min())[0][0] 
temp=abs(zvec-2*sigmaz)
zix_2sig = np.where(temp == temp.min())[0][0] 

temp=abs(xvec)
xix_0 = np.where(temp == temp.min())[0][0] 
temp=abs(xvec-1*sigmax)
xix_1sig = np.where(temp == temp.min())[0][0] 
temp=abs(xvec-2*sigmax)
xix_2sig = np.where(temp == temp.min())[0][0] 

print(WsConv.shape)
print(zix_0)
print(zix_1sig)
print(zix_2sig)
print(WsConv[zix_0,xix_0])
print(WsConv[zix_1sig,xix_0])
print(WsConv[zix_2sig,xix_0])
print(WsConv[zix_0,xix_1sig])
print(WsConv[zix_0,xix_2sig])
WsConv.__class__

In [ ]:
%%time
zvec, xvec, WsConv = Ws(gamma, rho, sigmaz, sigmax, dz, dx, 'oa')

In [ ]:
print(WsConv[zix_min,xix_min])
WsConv.__class__

In [ ]:
%%time 
zvec,xvec, WsConv = Ws(gamma,rho,sigmaz,sigmax,dz,dx,'fft')

In [ ]:
print(WsConv[zix_min,xix_min])
WsConv.__class__

## timing test

In [ ]:
%%timeit -n 3 -r 3
Ws(gamma, rho, sigmaz, sigmax, dz, dx, method='cupy', xp=cp, cupy_conv=cupy_conv)

In [ ]:
%%timeit -n 3 -r 3
zvec,xvec, WsConv = Ws(gamma,rho,sigmaz,sigmax,dz,dx,'oa')

# I/O

In [ ]:
# Find index where zvec returns one sigma_z
temp=abs(zvec-1*sigmaz)
zix_1sig = np.where(temp == temp.min())[0][0] 
print(zix_1sig)
print(zvec[zix_1sig])

In [ ]:
print(zvec.shape[0])
print(xvec.shape[0])

In [ ]:
size = 1000

# cp doesn't have savetxt function, so load it to npy first hten use np to write
cp.save('temp.npy',WsConv)
dat = np.load('temp.npy')
np.savetxt('WsConv_'+str(size)+'.txt',dat)

# Below is Scratch for now

In [ ]:
zvec_abs=abs(zvec)
zix_min = np.where(abs(zvec_abs) == zvec_abs.min())[0][0] # find the index for which xvec[index] is closest to zero
xvec_abs=abs(xvec)
xix_min = np.where(abs(xvec_abs) == xvec_abs.min())[0][0]

WsConv[zix_min,xix_min] # returns Ws(~0,~0)

In [ ]:
WsConv.__class__

In [ ]:
cp.save('t1.npy',WsConv)

In [ ]:
dat = np.load('t1.npy')
np.savetxt('text.txt',dat)

In [ ]:
np.save('test',WsConv)
np.load('test.npy');

## Vectorize from Chris

In [ ]:
@np.vectorize
def mf(a,b):
    return lambda_p_Gauss(a,b)
vf = np.vectorize(mf)
vf([[0.00001,0.00002],[0.00003,0.00004]] , [[0.00001,0.00002],[0.0003,0.00004]])

In [ ]:
zvec_abs=abs(zvec)
zix_min = np.where(abs(zvec_abs) == zvec_abs.min())[0][0] # find the index for which xvec[index] is closest to zero
xvec_abs=abs(xvec)
xix_min = np.where(abs(xvec_abs) == xvec_abs.min())[0][0]

plt.plot(zvec/sigmaz, WsConv[:,int(xix_min)]/1e6, '-', color='black', label=f"x = {xvec[xix_min]/sigmax:5.2f} $\sigma_x$");
plt.legend(loc='upper left')
plt.xlabel('z/$\sigma_z$')
plt.ylabel('Ws$(10^6 m^{-2})$')

In [ ]:
%timeit -n 3 -r 4 ndimage.convolve(a, k, mode='constant', cval=0.0)
%timeit -n 3 -r 4 signal.convolve2d(a, k, mode='same')
%timeit -n 3 -r 4 fftconvolve(a, k, mode='same')
%timeit -n 3 -r 4 oaconvolve(a, k, mode='same')

a = cp.random.randint(5, size=(N, N))
k = cp.random.randint(5, size=(2*N, 2*N))
%timeit -n 3 -r 4 filters.convolve(a, k, mode='constant', cval=0.0)

In [ ]:
from scipy.ndimage import convolve as conv
from scipy.signal import convolve2d, fftconvolve, oaconvolve
a = np.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = np.array([[1,1,1,1,1],[1,1,1,1,0],[1,1,1,0,0],[1,1,0,0,0],[1,0,0,0,0]])
print(conv(a, k, mode='constant', cval=0.0))

In [ ]:
a = cp.array([[1, 2, 0, 0],[5, 3, 0, 4],[0, 0, 0, 7],[9, 3, 0, 0]])
k = cp.array([[1,1,1,1,1],[1,1,1,1,0],[1,1,1,0,0],[1,1,0,0,0],[1,0,0,0,0]])
print(cupy_conv(a, k, mode='constant', cval=0.0))